In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import re

from nltk.corpus import stopwords
stop = list(set(stopwords.words('english')))

In [2]:
f_train = open('../data/train_14k_split_conll.txt','r',encoding='utf8')
line_train = f_train.readlines()

f_val = open('../data/dev_3k_split_conll.txt','r',encoding='utf8')
line_val = f_val.readlines()

f_test = open('../data/Hindi_test_unalbelled_conll_updated.txt','r',encoding='utf8')
line_test = f_test.readlines()

In [3]:
def get_data(lines):
    doc_count = 0
    i = 0
    #global train_word, train_word_type, train_doc_sentiment, train_uid, train_doc_id
    train_word = []
    train_word_type = []
    train_doc_sentiment = []
    train_uid = []
    train_doc_id = []

    while i < len(lines):
        '''
        if i < len(lines) and lines[i].split('\t')[0] in ['http','https']:
            i += 1
            while i < len(lines) and lines[i].split('\t')[0] != '/':
                i += 1
                
            i += 1    
        ''' 

        if i < len(lines) and lines[i].split('\t')[0] in ['@']:
            i += 2

        if i < len(lines):
            line = lines[i]
            line = line.replace('\n','').strip().lower()
            if line.split('\t')[0] == 'meta':
                doc_count += 1
                uid = int(line.split('\t')[1])
                sentiment = line.split('\t')[2]
                #if doc_count == 1575:
                #    print (uid)
                i += 1

            elif len(line.split('\t')) >= 2:
                if line.split('\t')[0] not in stop:
                    train_uid.append(uid)
                    train_doc_sentiment.append(sentiment)
                    train_word.append(line.split('\t')[0])
                    train_word_type.append(line.split('\t')[1])
                    train_doc_id.append(doc_count)

                i += 1

            else:
                i += 1

    train_df = pd.DataFrame()
    train_df['doc_id'] = train_doc_id
    train_df['word'] = train_word
    train_df['word_type'] = train_word_type
    train_df['uid'] = train_uid
    train_df['sentiment'] = train_doc_sentiment
    
    return train_df

In [4]:
def get_data_test(lines):
    doc_count = 0
    i = 0
    #global train_word, train_word_type, train_doc_sentiment, train_uid, train_doc_id
    train_word = []
    train_word_type = []
    train_uid = []
    train_doc_id = []

    while i < len(lines):
        '''
        if i < len(lines) and lines[i].split('\t')[0] in ['http','https']:
            i += 1
            while i < len(lines) and lines[i].split('\t')[0] != '/':
                i += 1
                
            i += 1    
        ''' 

        if i < len(lines) and lines[i].split('\t')[0] in ['@']:
            i += 2

        if i < len(lines):
            line = lines[i]
            line = line.replace('\n','').strip().lower()
            if line.split('\t')[0] == 'meta':
                doc_count += 1
                uid = int(line.split('\t')[1])
                #if doc_count == 1575:
                #    print (uid)
                i += 1

            elif len(line.split('\t')) >= 2:
                if line.split('\t')[0] not in stop:
                    train_uid.append(uid)
                    train_word.append(line.split('\t')[0])
                    train_word_type.append(line.split('\t')[1])
                    train_doc_id.append(doc_count)

                i += 1

            else:
                i += 1

    train_df = pd.DataFrame()
    train_df['doc_id'] = train_doc_id
    train_df['word'] = train_word
    train_df['word_type'] = train_word_type
    train_df['uid'] = train_uid
    
    return train_df

In [5]:
train_df = get_data(line_train)
val_df = get_data(line_val)
test_df = get_data_test(line_test)

In [6]:
train_df.shape, val_df.shape, test_df.shape

((289253, 5), (62618, 5), (61735, 4))

In [7]:
train_df.uid.nunique(), val_df.uid.nunique(), test_df.uid.nunique()

(14000, 3000, 3000)

In [8]:
train_df = train_df[train_df.word != 'http']
train_df = train_df[train_df.word != 'https']
train_df = train_df[train_df.word_type != 'o']
print (train_df.shape)

val_df = val_df[val_df.word != 'http']
val_df = val_df[val_df.word != 'https']
val_df = val_df[val_df.word_type != 'o']
print (val_df.shape)

test_df = test_df[test_df.word != 'http']
test_df = test_df[test_df.word != 'https']
test_df = test_df[test_df.word_type != 'o']
print (test_df.shape)

(223215, 5)
(48339, 5)
(47683, 4)


In [9]:
train_df.word = train_df.word.apply(lambda x: re.sub("[^a-zA-Z0-9]", "",x))
train_df = train_df[train_df.word.str.len() >= 3]
print (train_df.shape)

val_df.word = val_df.word.apply(lambda x: re.sub("[^a-zA-Z0-9]", "",x))
val_df = val_df[val_df.word.str.len() >= 3]
print (val_df.shape)

test_df.word = test_df.word.apply(lambda x: re.sub("[^a-zA-Z0-9]", "",x))
test_df = test_df[test_df.word.str.len() >= 3]
print (test_df.shape)

(175903, 5)
(38082, 5)
(37583, 4)


In [10]:
all_words = set(pd.concat([train_df[['word']],val_df[['word']],test_df[['word']]], axis=0).word)
print ("Total number of words {}".format(len(all_words)))

Total number of words 51515


In [11]:
all_words = pd.concat([train_df[['word']],val_df[['word']],test_df[['word']]], axis=0)
all_words = all_words.word.value_counts().reset_index()
all_words.columns = ['word','tot_count']
top_words = all_words[all_words.tot_count >= 2]
print (top_words.shape)
print (top_words.head(5))

(16771, 2)
   word  tot_count
0   hai       5902
1  nahi       3744
2   bhi       1965
3   aur       1799
4  modi       1415


In [12]:
all_bert_words = pd.DataFrame(['[PAD]','[UNK]','[CLS]','[SEP]'],columns=['word'])
all_bert_words = pd.concat([all_bert_words,top_words[['word']].drop_duplicates().reset_index(drop=True)],axis=0)
all_bert_words.to_csv("../bert_vocab.txt",index=False,header=False)

In [13]:
train_texts = train_df.groupby(['uid'],sort=True)['word'].apply(lambda x: " ".join(x)).reset_index()
train_texts = pd.merge(train_texts,train_df[['uid','sentiment']],how='left').drop_duplicates().reset_index(drop=True)
train_texts.columns = ['uid','text','sentiment']
#train_texts.text = train_texts.text.apply(lambda x: '[CLS] ' + x + ' [SEP]')

val_texts = val_df.groupby(['uid'],sort=True)['word'].apply(lambda x: " ".join(x)).reset_index().reset_index(drop=True)
val_texts = pd.merge(val_texts,val_df[['uid','sentiment']],how='left').drop_duplicates()
val_texts.columns = ['uid','text','sentiment']
#val_texts.text = val_texts.text.apply(lambda x: '[CLS] ' + x + ' [SEP]')

test_texts = test_df.groupby(['uid'],sort=True)['word'].apply(lambda x: " ".join(x)).reset_index().reset_index(drop=True)
test_texts.columns = ['uid','text']
#test_texts.text = test_texts.text.apply(lambda x: '[CLS] ' + x + ' [SEP]')


In [14]:
train_texts.head(5)

,uid,text,sentiment
0,8,love looks good maddie ako lang yung sobrang m...,neutral
1,14,min lyching manakgupta mein kahna nae chahta q...,neutral
2,26,best luck sir world cup liye bhot bhot subhkam...,positive
3,27,yes great dialogues one also chupke chupke chh...,positive
4,33,tarekfatah tere baap liye bola kya tha bhadwe ...,negative


In [15]:
test_texts.head(5)

,uid,text
0,1,keh aese rahe jaise pakistan wale karte south ...
1,4,anus prerna way ran saving ram ram jay rajaraa...
2,17,shukar hai pathan nae warna ptm nay wae dramay...
3,18,harsh pen decision much option arm big second ...
4,34,keep saying kenyan rugby beautiful save kru dr...


In [16]:
all_texts = pd.concat([train_texts[['uid','text','sentiment']],val_texts[['uid','text','sentiment']]],axis=0)
all_texts.sentiment.value_counts()

neutral     6392
positive    5616
negative    4992
Name: sentiment, dtype: int64

In [17]:
from transformers import BertTokenizer, BertConfig, BertModel
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

I0226 21:23:37.660035 4645359040 file_utils.py:38] PyTorch version 1.2.0 available.
Using TensorFlow backend.


In [18]:
tokenizer = BertTokenizer.from_pretrained('../bert_vocab.txt')

I0226 21:23:49.973178 4645359040 tokenization_utils.py:335] Model name '../bert_vocab.txt' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming '../bert_vocab.txt' is a path, a model identifier, or url to a directory containing tokenizer files.
W0226 21:23:49.974266 4645359040 tokenization_utils.py:347] Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
I0226 21:23:49.975219 4645359040 tokenization_utils.py:416] loading f

In [19]:
max_len = 20

In [20]:
n_output = all_texts.sentiment.nunique() #number of possible outputs
senti_dict = {'negative':0,'neutral':1,'positive':2}
in_senti_dict = {0:'negative',1:'neutral',2:'positive'}
all_texts.sentiment = all_texts.sentiment.apply(lambda x: senti_dict[x])

In [21]:
train_tokens = []
for text in tqdm(all_texts.text.values.tolist()):
    train_tokens += [tokenizer.encode(text,add_special_tokens=False)]

100%|██████████| 17000/17000 [00:04<00:00, 3798.18it/s]


In [22]:
test_tokens = []
for text in tqdm(test_texts.text.values.tolist()):
    test_tokens += [tokenizer.encode(text,add_special_tokens=False)]

100%|██████████| 3000/3000 [00:00<00:00, 3306.15it/s]


In [23]:
train_tokens_padded = []
train_attention_mask = []
train_seg_ids = []

for tokens in tqdm(train_tokens):
    tokens = tokens[:max_len]
    token_len = len(tokens)
    one_mask = [1]*token_len
    zero_mask = [0]*(max_len-token_len)
    padded_input = tokens + zero_mask
    attention_mask = one_mask + zero_mask
    
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == 3:
            current_segment_id = 1
    segments = segments + [0] * (max_len - len(tokens))
    
    train_tokens_padded += [padded_input]
    train_attention_mask += [attention_mask]
    train_seg_ids += [segments]

100%|██████████| 17000/17000 [00:00<00:00, 191819.56it/s]


In [24]:
test_tokens_padded = []
test_attention_mask = []
test_seg_ids = []

for tokens in tqdm(test_tokens):
    tokens = tokens[:max_len]
    token_len = len(tokens)
    one_mask = [1]*token_len
    zero_mask = [0]*(max_len-token_len)
    padded_input = tokens + zero_mask
    attention_mask = one_mask + zero_mask
    
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == 102:
            current_segment_id = 1
    segments = segments + [0] * (max_len - len(tokens))
    
    test_tokens_padded += [padded_input]
    test_attention_mask += [attention_mask]
    test_seg_ids += [segments]

100%|██████████| 3000/3000 [00:00<00:00, 192431.63it/s]


In [25]:
print (train_tokens_padded[0], train_attention_mask[0], train_seg_ids[0])

[13, 865, 17, 1, 2223, 2737, 2918, 9642, 8725, 3591, 86, 1, 9033, 2145, 1, 1, 0, 0, 0, 0] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [26]:
train_output = torch.LongTensor(to_categorical(all_texts.sentiment))

In [27]:
train_tokens_padded = torch.LongTensor(np.asarray(train_tokens_padded))
train_attention_mask = torch.LongTensor(np.asarray(train_attention_mask))
train_seg_ids = torch.LongTensor(np.asarray(train_seg_ids))
test_tokens_padded = torch.LongTensor(np.asarray(test_tokens_padded))
test_attention_mask = torch.LongTensor(np.asarray(test_attention_mask))
test_seg_ids = torch.LongTensor(np.asarray(test_seg_ids))

In [28]:
print (train_tokens_padded.shape, train_attention_mask.shape, train_seg_ids.shape, test_tokens_padded.shape, test_attention_mask.shape, test_seg_ids.shape)


torch.Size([17000, 20]) torch.Size([17000, 20]) torch.Size([17000, 20]) torch.Size([3000, 20]) torch.Size([3000, 20]) torch.Size([3000, 20])


In [29]:
dev_tokens_padded = train_tokens_padded[train_texts.shape[0]:]
train_tokens_padded = train_tokens_padded[:train_texts.shape[0]]

dev_attention_mask = train_attention_mask[train_texts.shape[0]:]
train_attention_mask = train_attention_mask[:train_texts.shape[0]]

dev_seg_ids = train_seg_ids[train_texts.shape[0]:]
train_seg_ids = train_seg_ids[:train_texts.shape[0]]

dev_output = train_output[train_texts.shape[0]:]
train_output = train_output[:train_texts.shape[0]]

In [30]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, accuracy_score

In [31]:
batch_size = 128

In [32]:
train_data = TensorDataset(train_tokens_padded, train_output)
val_data = TensorDataset(dev_tokens_padded, dev_output)

#dataloader
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

In [33]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, nout, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, nout)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        #print (src.shape)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(torch.mean(output,1))
        return output
    
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=max_len):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0) #.transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        #print (self.pe.shape)
        x = x + self.pe
        return self.dropout(x)

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ntokens = tokenizer.vocab_size # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 4 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 8 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
nout = all_texts.sentiment.nunique()
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, nout, dropout)
epochs = 20

In [45]:
model

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=200, out_features=200, bias=True)
        )
        (linear1): Linear(in_features=200, out_features=200, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=200, out_features=200, bias=True)
        (norm1): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=200, out_features=200, bias=True)
        )
        (linear1): Linear(in_features=200, out_features=20

In [46]:
print ("Total number of parameters to learn {}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

Total number of parameters to learn 4323603


In [47]:
optimizer = torch.optim.Adam(model.parameters(),lr=.001)
criterion = torch.nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [48]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    if device == 'cpu':
        rounded_preds = preds.detach().numpy().argmax(1)
        rounded_correct = y.detach().numpy().argmax(1)
    else:
        rounded_preds = preds.detach().cpu().numpy().argmax(1)
        rounded_correct = y.detach().cpu().numpy().argmax(1)
    
    return accuracy_score(rounded_correct,rounded_preds)

In [49]:
def f1_torch(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    if device == 'cpu':
        rounded_preds = preds.detach().numpy().argmax(1)
        rounded_correct = y.detach().numpy().argmax(1)
    else:
        rounded_preds = preds.detach().cpu().numpy().argmax(1)
        rounded_correct = y.detach().cpu().numpy().argmax(1)
    
    return f1_score(rounded_correct,rounded_preds,average='macro')

In [50]:
def train(model, train_loader, optimizer, criterion):
    global predictions, labels, loss
    
    epoch_loss = 0
    epoch_acc = 0
    f1_scores = 0
    
    model.train()
    
    counter = 0
    for tokens, labels in tqdm(train_loader):
        
        counter += 1
        
        optimizer.zero_grad()
        
        predictions = model(tokens) #.squeeze(1)
        predictions = torch.softmax(predictions,dim=-1)
        
        #loss = criterion(predictions, labels)
        loss = criterion(predictions, torch.max(labels, 1)[1])
        
        acc = binary_accuracy(predictions, labels)
        f1_score_batch = f1_torch(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc
        f1_scores += f1_score_batch
        
    return epoch_loss / counter, epoch_acc / counter, f1_scores/counter

In [51]:
def evaluate(model, val_loader, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    f1_scores = 0
    
    model.eval()
    
    counter = 0
    with torch.no_grad():
        for tokens, labels in tqdm(val_loader):

            counter += 1


            predictions = model(tokens) #.squeeze(1)
            predictions = torch.softmax(predictions,dim=-1)
            
            #loss = criterion(predictions, labels)
            loss = criterion(predictions, torch.max(labels, 1)[1])

            acc = binary_accuracy(predictions, labels)
            f1_score_batch = f1_torch(predictions, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc
            f1_scores += f1_score_batch
        
    return epoch_loss / counter, epoch_acc / counter, f1_scores/counter

In [52]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [53]:
best_valid_loss = 999

for epoch in range(epochs):
    
    start_time = time.time()
    
    train_loss, train_acc, train_f1 = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc, valid_f1 = evaluate(model, val_loader, criterion)
       
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '../models/model_transformer.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train F1: {train_f1*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val F1: {valid_f1*100:.2f}%')

  0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 1m 26s
	Train Loss: 1.076 | Train Acc: 44.01% | Train F1: 31.11%
	 Val. Loss: 1.066 |  Val. Acc: 45.55% | Val F1: 35.35%


  0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 02 | Epoch Time: 1m 25s
	Train Loss: 0.961 | Train Acc: 56.94% | Train F1: 52.26%
	 Val. Loss: 0.983 |  Val. Acc: 53.86% | Val F1: 53.46%


  0%|          | 0/110 [00:00<?, ?it/s]

...

Epoch: 18 | Epoch Time: 1m 38s
	Train Loss: 0.650 | Train Acc: 90.42% | Train F1: 90.36%
	 Val. Loss: 0.990 |  Val. Acc: 54.83% | Val F1: 54.90%


  0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 19 | Epoch Time: 1m 30s
	Train Loss: 0.646 | Train Acc: 90.76% | Train F1: 90.71%
	 Val. Loss: 0.988 |  Val. Acc: 55.42% | Val F1: 55.41%


100%|██████████| 24/24 [00:03<00:00,  7.37it/s]

Epoch: 20 | Epoch Time: 1m 27s
	Train Loss: 0.643 | Train Acc: 90.99% | Train F1: 90.94%
	 Val. Loss: 0.994 |  Val. Acc: 54.56% | Val F1: 54.63%
